In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import math

def foldChange(diseased, control):
    return diseased/control

def log2foldChange(foldChange):
    return math.log(foldChange, 2)

dat = pd.read_table('test2.txt', sep='\t')

controls = []
parkinsons = []
columns = list(dat)
for column in columns:
    if column.startswith('C_'):
        controls.append(column)
    elif column.startswith('P_'):
        parkinsons.append(column)

baseMean = []
contExpMeans = []
parExpMeans = []
foldChanges = []
log2foldChanges = []
contStdErrors = []
parStdErrors = []

for i in range(len(dat.index)):
    mean = np.mean(dat.ix[i,2:])
    contExpMean = np.mean(dat[controls].ix[i,:])
    parExpMean = np.mean(dat[parkinsons].ix[i,:])
    foldCh = foldChange(parExpMean, contExpMean)
    log2foldCh = log2foldChange(foldCh)
    contStdErr = stats.sem(dat[controls].ix[i,:])
    parStdErr = stats.sem(dat[parkinsons].ix[i,:])
    contStdErrors.append(contStdErr)
    parStdErrors.append(parStdErr)
    log2foldChanges.append(log2foldCh)
    foldChanges.append(foldCh)
    parExpMeans.append(parExpMean)
    contExpMeans.append(contExpMean)
    baseMean.append(mean)

    

out = {'EnsemblID':dat['EnsemblID'],
       'genes': dat['symbol'],
      'baseMean': baseMean,
      'contExpMean': contExpMeans,
      'parExpMean': parExpMeans,
      'foldCh': foldChanges,
      'log2fCh': log2foldChanges,
      'contSE': contStdErrors,
      'parSE': parStdErrors}

outDF = pd.DataFrame(out, columns=['EnsemblID', 'genes', 'baseMean', 'contExpMean', 'parExpMean', 'foldCh', 'log2fCh',
                                  'contSE', 'parSE'])
sortedOut = outDF.sort_values(by='foldCh', ascending=False)

#sortedOut.to_csv('parkOut.txt', sep='\t')